In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urljoin
import re

In [2]:
bank_name = "Jordan Commercial Bank"
base_url = "https://www.jcbank.com.jo"
url = f"{base_url}/ar"
headers = {"User-Agent": "Mozilla/5.0"}

In [3]:
folder_name = "Jordan_Commercial_Bank_Images"
os.makedirs(folder_name, exist_ok=True)

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")


In [4]:
slides = soup.select(".jcb-state-slider .jcb-state-item")

data = []

for slide in slides:
    style = slide.get("style", "")
    match = re.search(r"url\(['\"]?(.*?)['\"]?\)", style)
    if not match:
        continue

    img_src = match.group(1)
    full_img_url = urljoin(base_url, img_src)
    img_name = img_src.split("/")[-1]
    img_path = os.path.join(folder_name, img_name)

    # محاولة تحميل الصورة
    try:
        img_response = requests.get(full_img_url, stream=True)
        img_response.raise_for_status()
        with open(img_path, "wb") as f:
            f.write(img_response.content)
        print(f"✔️ تم تحميل الصورة: {img_name}")
    except Exception as e:
        print(f"❌ فشل تحميل الصورة: {img_name} - {e}")

    # رابط المقال (أو الرابط العام)
    parent_a = slide.find_parent("a")
    if parent_a and parent_a.has_attr("href") and parent_a["href"].strip():
        article_url = urljoin(base_url, parent_a["href"])
    else:
        article_url = url

    data.append({
        "Bank_Name": bank_name,
        "Image_Name": img_name,
        "Image_Link": full_img_url,
        "Article_Link": article_url
    })


✔️ تم تحميل الصورة: jcb250415budgetad.jpg


In [7]:
df = pd.DataFrame(data)
df

,Bank_Name,Image_Name,Image_Link,Article_Link
0,Jordan Commercial Bank,jcb250415budgetad.jpg,https://www.jcbank.com.jo/uploads/2025/04/jcb2...,https://www.jcbank.com.jo/ar
